In [2]:
#KERAS_BACKEND="theano"
import numpy as np
import keras
from keras.utils.np_utils import to_categorical
from keras.models import model_from_json

class neural_chatbot():
    
    def __init__(self):
        #self.initialize()
        self.load_model()
        self.sys_prediction=
    def save_model(self):
        # serialize model to JSON
        model_json = self.model_ABA.to_json()
        with open("model_ABA.json", "w") as json_file:
            json_file.write(self.model_json)
        # serialize weights to HDF5
        self.model_ABA.save_weights("model_ABA.h5")
        print("Saved model to disk")
    
    def load_model(self):
        # load json and create model
        json_file = open('model_ABA.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        self.model_ABA = model_from_json(loaded_model_json)
        # load weights into new model
        self.model_ABA.load_weights("model_ABA.h5")
        print("Loaded model from disk")
        
    def initialize(self):
        
        self.model_A = Sequential()
        self.model_A.add(LSTM(embed_dim,input_shape=(max_sent_length,embed_dim),return_sequences=True, dropout_W=0.2, dropout_U=0.2))

        self.model_A.compile(loss='cosine_proximity',
              optimizer='rmsprop',
              metrics=['accuracy']
              )
        self.model_A.load_weights('model_A_weights.h5')
        
        self.model_B = Sequential()
        self.model_B.add(LSTM(embed_dim,input_shape=(max_sent_length,embed_dim),return_sequences=True, dropout_W=0.2, dropout_U=0.2))

        self.model_B.compile(loss='cosine_proximity',
              optimizer='rmsprop',
              metrics=['accuracy']
              )
        self.model_B.load_weights('model_B_weights.h5')
        
        self.model_ABA = Sequential()
        self.model_ABA.add(Merge([model_A, model_B], mode = 'concat'))
        self.model_ABA.add(LSTM(embed_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
        self.model_ABA.add(LSTM(embed_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
        #model_ABA.add(TimeDistributed(Dense(vocab_size,activation='softmax')))

        self.model_ABA.compile(loss='cosine_proximity',
              optimizer='rmsprop',
              metrics=['accuracy']
              )
        self.model_ABA.load_weights('model_ABA_weights.h5')
        
    def reinforcement_learn(user_emotion,sys_response):
        total_emo = user_emotion[0] + user_emotion[1]
        sys_res = np.argmax(sys_response, axis=3)
        if total_emo > 0:
            #sys_res = (10*sys_res)+10
            sys_res_categorical=to_categorical(sys_res.astype(int),20).reshape((1,300,-1,20))
            self.model_ABA.fit(self.user_input,sys_res_categorical)
        else:
            sys_res_categorical=to_categorical(sys_res.astype(int),20).reshape((1,300,-1,20))*(-1)
            self.model_ABA.fit(self.user_input,sys_res_categorical)
        self.save_model()
        self.load_model()
        
    def convert3d_to_4d(sys_response3d):
        sys_res3d_standardized = (10*sys_res)+10
        sys_response = to_categorical(sys_res3d_standardized.astype(int),20).reshape((1,300,-1,20))

    def model_response(User_B):
        #User_A1 = "5 5 1 @I really love you. I think I want to marry you!"
        #User_B = "1 5 5 @yeah well I hate you and never want to see you again! "
        #User_A1 = w2v.vectorize( w2v_model, User_A1 , pad_length = max_sent_length) 
        User_B  = w2v.vectorize( w2v_model, User_B, pad_length = max_sent_length) 
        reinforcement_learn(self.sys_prediction)
        self.user_input = [ np.array( [self.sys_prediction]), np.array([User_B]) ]
        prediction = self.model_ABA.predict(user_input)
        sys_pred = np.argmax(prediction, axis=3)
        self.sys_prediction = (sys_pred-10)/10
        return self.most_likely_sent( self.sys_prediction[0], metadata['w2idx'].keys(),w2v_model)
    
    def cosign_similarity( a, b):
        dot = np.dot(a,b)
        norm = np.linalg.norm(a-b)
        return( dot/norm )

    def most_likely_word( prediction, words, w2v_model ):
        ret = ""
        max_so_far = self.cosign_similarity( np.array([1]+[0]*299),prediction ) #must be better than blank
        #Best word in dictionary
        for word in words:
            similarity = cosign_similarity( w2v.vectorize(w2v_model,word)[0], prediction )
            if similarity > max_so_far:
                max_so_far = similarity
                ret = word    
        return( ret )

    def most_likely_sent( prediction_vec, words, w2v_model ):
        predicted_words = []
        for prediction in prediction_vec:
            predicted_words.append( most_likely_word(prediction,words,w2v_model))
        ret = " ".join(predicted_words)
        return( ret )

In [25]:
## Credit to http://sentistrength.wlv.ac.uk/documentation/Python.txt for the code
 
import shlex, subprocess
 
def RateSentiment(sentiString):
    #open a subprocess using shlex to get the command line string into the correct args list format
    p = subprocess.Popen(shlex.split("java -jar C:/Users/anul/Downloads/SentiStrength.jar stdin sentidata \
                                     C:/Users/anul/Downloads/Sentstrength_Data_Sept2011/ "),
                         stdin=subprocess.PIPE,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    #communicate via stdin the string to be rated. Note that all spaces are replaced with +
    stdout_text, stderr_text = p.communicate(sentiString.replace(" ","+"))
    #remove the tab spacing between the positive and negative ratings. e.g. 1    -5 -> 1-5
    stdout_text = stdout_text.rstrip().replace("\t","")
     

    return stdout_text
print(RateSentiment("I hate you"))

OSError: [WinError 6] The handle is invalid

In [17]:
p=RateSentiment('hate')
print(p)

ValueError: No closing quotation

In [23]:
from subprocess import *

def jarWrapper(*args):
    process = Popen(['java', '-jar']+list(args), stdout=PIPE, stderr=PIPE)
    ret = []
    while process.poll() is None:
        line = process.stdout.readline()
        if line != '' and line.endswith('\n'):
            ret.append(line[:-1])
    stdout, stderr = process.communicate()
    ret += stdout.split('\n')
    if stderr != '':
        ret += stderr.split('\n')
    ret.remove('')
    return ret

args = ['C:/Users/anul/Downloads/SentiStrength.jar', 'text', 'hate','sentidata', 'C:/Users/anul/Downloads/SentStrength_Data_Sept2011/'] # Any number of args to be passed to the jar file

result = jarWrapper(*args)

print(result)

OSError: [WinError 6] The handle is invalid